In [1]:
import pickle
import multiprocessing
import os, re, sys
import nltk
import csv
import gensim.models.word2vec as w2v
import glob2
import time

# word2vec with Reddit corpus

In [9]:
songdata_filepath = "songdata.csv"                #simple csv

reddit_filepath = "reddit-top-2.5-million-master/data" #this is a dir, we'll walk through it to create a
                                                  #dict of subreddits and posts


## just playing arond with etl on the song data

In [10]:
songs_list = []

with open(songdata_filepath, 'rb') as csvfile:
    song_reader = csv.reader(csvfile)
    i = 0
    for row in song_reader:
        if(i == 0):
            next
        else:
            song = {'artist': row[0], 'title': row[1], 'lyric_link': row[2], 'text': row[3]}
            songs_list.append(song)
        i += 1

In [11]:
songs_list[258]

{'artist': 'Air Supply',
 'lyric_link': '/a/air+supply/do+it+again_20004902.html',
 'text': "I used to watch you walk away  \nTil the road you take would bend  \nI was the man to turn you on  \nAnd you knew it after all  \nCome tomorrow when I'm all weary  \nYou would turn me down again  \nAt least that's the way I can remember  \n  \nDo it again, do it again  \nDo it again, do it again  \n  \nEvery little word you told me got home  \nI'm still thinking about them  \nEvery little secret we could have known  \nAnd we can't live without them  \n  \nYou surely started something  \nYou surely gave me something new  \n  \nDo it again, do it again  \nDo it again, do it again  \n  \nAnd I do think about you each now and then  \nAnd each time it gets tighter  \nFrom the road I can see the last break  \nAnd my head's getting lighter  \n  \nDo it again, do it again (repeat)  \n\n",
 'title': 'Do It Again'}

In [12]:
len(songs_list)   #should be 57,650 songs

57650

## Now on to the subreddit posts

In [40]:
subreddit_list = {}

path = reddit_filepath + '/**'
files = glob2.glob(path)

len(files)   #should be 2,499 subreddits

subreddit_id_to_title = {}

In [53]:
j = 0
for file in files:
    subreddit = file.replace(reddit_filepath + "/", "").replace(".csv", "")
    subreddit_id_to_title[j] = subreddit
    j += 1
    with(open(file, 'rb')) as csvfile:
        reddit_reader = csv.reader(csvfile)
        posts = []
        reddit_reader.next()
        for post in reddit_reader:
            posts.append({'score': post[1].decode('utf-8'), 'title': post[4].decode('utf-8'), 'text': post[9].decode('utf-8')})
        
        subreddit_list[subreddit] = posts

In [54]:
subreddit_list["0x10c"][0]

{'score': u'430',
 'text': u"If you saw my tweet yesterday about being single now, that's the reason I haven't had that much inspiration lately to work on a game as big as 0x10c, and hopefully it explains why I haven't spoken much about the reason.\n\nI don't want to sound whiny on twitter, but I thought this excellent subreddit might want to know. I haven't forgotten about you!",
 'title': u'The reason for slow'}

In [58]:
reddit_corpus = ""
offset = 2495                                             #testing param only. this leaves you with the first 5 
                                                          #subreddits, with 1000 posts each for 5k posts
for i in range(0, len(subreddit_list) - 1 - offset):
    
    subreddit = subreddit_list[subreddit_id_to_title[i]]
    for post in subreddit:
        string_ripped_out = post['title'] + " " + post['text']
        reddit_corpus += string_ripped_out

In [59]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sentences = tokenizer.tokenize(reddit_corpus)

In [60]:
def sentence_to_wordlist(raw):
    """
    convert to list of words
    remove unnecessary, split into words, no hyphens
    list of words
    """
    clean = re.sub("[^a-zA-z]", " ", raw)
    words = clean.split()
    return words

sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

token_count = sum([len(sentence) for sentence in sentences])
print(token_count)

263163


In [61]:
reddit_vec = w2v.Word2Vec(
            sg=1,
            seed=1337,
            workers=4,
            size=300,
            min_count=3,
            window=7,
            sample=0.0001
        )

reddit_vec.build_vocab(sentences)

In [63]:
reddit_vec.train(sentences, total_examples=len(sentences), epochs=10)

1194365

In [65]:
reddit_vec.most_similar("youtube")

[(u'watch', 0.9908463954925537),
 (u'feature', 0.9521093964576721),
 (u'page', 0.9492238759994507),
 (u'search', 0.9221713542938232),
 (u'result', 0.9204763174057007),
 (u'[this]', 0.9118918180465698),
 (u'com', 0.9076484441757202),
 (u'screenshot', 0.9025120735168457),
 (u'jpg', 0.9006019830703735),
 (u'images', 0.899824857711792)]